# TOOL : List of Exposures  at USDF in repo/main or repo/embargo

- work with Weakly_2025_36
- use jupyter kernel LSST
- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2025/09/15
- last update : 2025/09/16
- last update : 2025/09/18 : handle repo/embargo

- Extract exposure from Butler
- select those with holo
- generate output file with information from butler registry

In [ ]:
import sys
print(sys.executable)

In [ ]:
print(sys.path)

In [ ]:
! eups list -s | grep LOCAL

In [ ]:
! echo $IMAGE_DESCRIPTION
! eups list -s lsst_distrib

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib.colors import LogNorm

from mpl_toolkits.axes_grid1 import make_axes_locatable

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.io import fits
from astropy.time import Time

In [ ]:
from IPython.display import display, HTML

In [ ]:
#import lsst.afw.display as afwDisplay
#afwDisplay.setDefaultBackend('matplotlib')

In [ ]:
import lsst.daf.butler as dafButler

In [ ]:
FLAG_REPO_EMBARGO = True
FLAG_OUTPUT_PERFILTER = False

In [ ]:
if FLAG_REPO_EMBARGO:
    repo="/repo/embargo"
else:
    repo="/repo/main"

reponame = repo.replace("/","_")
print(reponame)
butler = dafButler.Butler(repo)
registry = butler.registry

In [ ]:
instrument = "LATISS"
date_start = 20220301
date_selection = 20220301
skymapName = "latiss_v1"
where_clause = "instrument = '" + instrument + "'"
where_clause_date = where_clause + f"and day_obs >= {date_start}"

In [ ]:
print(butler.registry.dimensions["exposure"].RecordClass.fields)

In [ ]:
collection='LATISS/raw/all'

## List of Exposures

In [ ]:
columns = [
    "id",
    "obs_id",
    "day_obs",
    "seq_num",
    "time_start",
    "time_end",
    "type",
    "target",
    "filter",
    "zenith_angle",
    "expos",
    "ra",
    "dec",
    "skyangle",
    "azimuth",
    "zenith",
    "science_program",
    "jd",
    "mjd",
]

In [ ]:
df_exposure = pd.DataFrame(
    {
        "id": pd.Series(dtype="int"),
        "obs_id": pd.Series(dtype="int"),
        "day_obs": pd.Series(dtype="int"),
        "seq_num": pd.Series(dtype="int"),
        "time_start": pd.Series(dtype="str"),  # ou 'datetime64[ns]' si c’est un datetime
        "time_end": pd.Series(dtype="str"),  # idem
        "type": pd.Series(dtype="str"),
        "target": pd.Series(dtype="str"),
        "filter": pd.Series(dtype="str"),
        "zenith_angle": pd.Series(dtype="float"),
        "expos": pd.Series(dtype="float"),  # ou 'int' selon le cas
        "ra": pd.Series(dtype="float"),
        "dec": pd.Series(dtype="float"),
        "skyangle": pd.Series(dtype="float"),
        "azimuth": pd.Series(dtype="float"),
        "zenith": pd.Series(dtype="float"),
        "science_program": pd.Series(dtype="str"),
        "jd": pd.Series(dtype="float"),
        "mjd": pd.Series(dtype="float"),
    }
)

In [ ]:
# save the data array in rows before saving in pandas dataframe
rows = []
for count, info in enumerate(registry.queryDimensionRecords("exposure", where=where_clause_date)):
    try:
        jd_start = info.timespan.begin.value
        jd_end = info.timespan.end.value
        the_Time_start = Time(jd_start, format="jd", scale="utc")
        the_Time_end = Time(jd_end, format="jd", scale="utc")
        mjd_start = the_Time_start.mjd
        mjd_end = the_Time_end.mjd
        isot_start = the_Time_start.isot
        isot_end = the_Time_end.isot

        if count == 0:
            print("===== Time Conversion Debug Info =====")
            print(f"JD start      : {jd_start} (type: {type(jd_start)})")
            print(f"JD end        : {jd_end} (type: {type(jd_end)})")
            print(f"MJD start     : {mjd_start} (type: {type(mjd_start)})")
            print(f"MJD end       : {mjd_end} (type: {type(mjd_end)})")
            print(f"ISOT start    : {isot_start} (type: {type(isot_start)})")
            print(f"ISOT end      : {isot_end} (type: {type(isot_end)})")
            print("=======================================")

        # put row in a dictionnary before stacking
        row = {
            "id": info.id,
            "obs_id": info.obs_id,
            "day_obs": info.day_obs,
            "seq_num": info.seq_num,
            "time_start": isot_start,
            "time_end": isot_end,
            "type": info.observation_type,
            "target": info.target_name,
            "filter": info.physical_filter,
            "zenith_angle": info.zenith_angle,
            "expos": info.exposure_time,  # Exemple : adapter selon ton objet
            "ra": info.tracking_ra,
            "dec": info.tracking_dec,
            "skyangle": info.sky_angle,
            "azimuth": info.azimuth,
            "zenith": info.zenith_angle,
            "science_program": info.science_program,
            "jd": float(jd_start),
            "mjd": float(mjd_start),
        }
        rows.append(row)

    except ValueError as e:
        print(f"Erreur de valeur : {e}")
    except FileNotFoundError as e:
        print(f"Fichier introuvable : {e}")
    except Exception as e:
        print(f"Erreur inattendue : {type(e).__name__} - {e}")
        print(f">>>   Unexpected error at row {count}:", sys.exc_info()[0])
        traceback.print_exc()  # affiche la stack trace complète

In [ ]:
# Création finale du DataFrame
df_exposure = pd.DataFrame(rows)

In [ ]:
df_exposure

In [ ]:
#df_exposure = df_exposure.astype({"id": int,'day_obs': int,'seq_num':int})

## Selection of science exposures

In [ ]:
df_science = df_exposure[df_exposure.type == 'science']

In [ ]:
df_science.sort_values(by="id",ascending=True,inplace=True)

In [ ]:
df_science.head(60)

In [ ]:
df_science.tail(60)

In [ ]:
df_science.day_obs.unique()

# List of filters

In [ ]:
list_of_filters = df_science['filter'].unique()
list_of_filters

In [ ]:
selected_filters = []
for filt in list_of_filters:
    #flag_sel = (filt.find('holo4') != -1) or (filt.find('ronchi90lpmm') != -1) or (filt.find('ronchi170lpmm') != -1)
    flag_sel = (filt.find('holo4') != -1) 
    #flag_sel = (filt.find('ronchi170') != -1) 
    if flag_sel:
        selected_filters.append(filt) 
selected_filters = np.array(selected_filters)
print(selected_filters)       

## Selection of filter

- But now we have a loop on filters

In [ ]:
#idx_sel_filt = 0
#selected_filter = selected_filters[0]
#selected_filter

## Final selection of exposure

- loop on selected filters

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
all_def_science_selected_final = []

for selected_filter in selected_filters:
    print(selected_filter)
    df_science_selected_final = df_science[df_science["filter"] ==  selected_filter]
    all_def_science_selected_final.append(df_science_selected_final.copy())
    

In [ ]:
if FLAG_OUTPUT_PERFILTER:
    for idx,selected_filter in enumerate(selected_filters):
        print(f"---------------{selected_filter}-----------------")
        df = all_def_science_selected_final[idx]
        print(df.head())
        filename_out_csv = "holosummary_" + selected_filter + reponame +".csv"
        filename_out_excel = "holosummary_" + selected_filter + reponame + ".xlsx"
        df.to_csv(filename_out_csv) 
        #! pip install openpyxl
        #df.to_excel(filename_out_excel) 
    

### Save all holo with filters

In [ ]:
df_allholo = pd.concat(all_def_science_selected_final)
df_allholo.sort_values(by="id",ascending=True,inplace=True)

In [ ]:
filename_out_csv = "holosummary_" + "all_filters" +  reponame +".csv"
df_allholo.to_csv(filename_out_csv) 